In [34]:
import cv2
URL = "http://192.168.29.220:8080/video"
FRAME_SIZE = (1280, 720)

RED = rgb(213, 48, 44)
BLUE = rgb(10, 137, 192)
GREEN = rgb(155, 202, 72)
YELLOW = rgb(220, 197, 83)
ORANGE = rgb(209, 89, 37)
WHITE = rgb(222, 218, 215)

In [11]:
REFERENCE_COLORS = {
    'R': (213, 48, 44),
    'B': (10, 137, 192),
    'G': (155, 202, 72),  # Same as blue in your data, usually slightly different
    'Y': (220, 197, 83),
    'O': (209, 89, 37),
    'W': (222, 218, 215),
}

### Reading Video

In [2]:
cam = cv2.VideoCapture("http://192.168.29.220:8080/video")

while True:
    ret, frame = cam.read()

    frame = cv2.resize(frame, (1280, 720)) #  720p
    cv2.imshow("Video", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/sweshik/Desktop/RubikSense/.linuxenv/lib/python3.12/site-packages/cv2/qt/plugins"


### Detecting the edges

In [18]:
cam = cv2.VideoCapture("http://192.168.29.220:8080/video")

while True:
    ret, frame = cam.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1280, 720))

    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur for noise reduction
    blurred_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred_frame, 50, 150) # Adjust thresholds as needed

    # Display frames
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Edge Detected Frame', edges)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

[mjpeg @ 0x26eef880] overread 8


### Capturing the face

In [19]:
import cv2
import numpy as np
import os

# Create output folder if it doesn't exist
os.makedirs("cube_faces", exist_ok=True)

cam = cv2.VideoCapture("http://192.168.29.220:8080/video")

while True:
    ret, frame = cam.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1280, 720))

    # Preprocessing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Apply morphological closing to connect edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)


    # Convert edges to BGR and overlay on original frame
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    overlay = cv2.addWeighted(frame, 0.6, edges_colored, 1.0, 0)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_square = None
    max_area = 0

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area:
            approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)
            if len(approx) == 4 and cv2.isContourConvex(approx):
                max_area = area
                largest_square = approx

    key = cv2.waitKey(1) & 0xFF

    if largest_square is not None:
        x, y, w, h = cv2.boundingRect(largest_square)

        # Draw the red rectangle on overlay
        cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Save the whole face when 'c' is pressed
        if key == ord('c'):
            face = frame[y:y+h, x:x+w]
            filename = f"cube_faces/face_{cv2.getTickCount()}.png"
            cv2.imwrite(filename, face)
            print("Saved cube face!")

    # Show the overlay with red bounding box
    cv2.imshow("Rubik Cube - Face Detection", overlay)

    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

### Seperating the cubelets

In [16]:
import cv2
import os

# Load the extracted face image
img = cv2.imread('cube_faces/face_73699184877594.png')  # Rename or use your file path
h, w, _ = img.shape

# Size of each cubelet
cubelet_h = h // 3
cubelet_w = w // 3

# Output folder to save cubelets
os.makedirs("cubelets", exist_ok=True)

count = 0
for i in range(3):
    for j in range(3):
        y1, y2 = i * cubelet_h, (i + 1) * cubelet_h
        x1, x2 = j * cubelet_w, (j + 1) * cubelet_w
        cubelet = img[y1:y2, x1:x2]
        cv2.imwrite(f'cubelets/cubelet_{count}.png', cubelet)
        count += 1

### Projecting Face and Cublets boundaries on video

In [16]:
import cv2
import numpy as np

def order_points(pts):
    # Order points in TL, TR, BR, BL order
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]     # top-left
    rect[2] = pts[np.argmax(s)]     # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect

def perspective_transform(face_image, face_contour):
    rect = order_points(face_contour.reshape(4, 2))
    face_size = 300  # size of the output face (300x300)
    dst_pts = np.array([
        [0, 0],
        [face_size - 1, 0],
        [face_size - 1, face_size - 1],
        [0, face_size - 1]
    ], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst_pts)
    warped = cv2.warpPerspective(face_image, M, (face_size, face_size))
    return warped, M, dst_pts, rect

def draw_cubelet_grid(warped):
    face_size = warped.shape[0]
    cell_size = face_size // 3

    # Draw thin black lines for cubelet boundaries
    for i in range(1, 3):
        cv2.line(warped, (0, i*cell_size), (face_size, i*cell_size), (0, 0, 0), 2)
        cv2.line(warped, (i*cell_size, 0), (i*cell_size, face_size), (0, 0, 0), 2)

    return warped

cap = cv2.VideoCapture("http://192.168.29.220:8080/video")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize for consistency
    frame = cv2.resize(frame, (1280, 720))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 150)

    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_area = 0
    best_contour = None

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > largest_area:
                largest_area = area
                best_contour = approx

    if best_contour is not None:
        # Draw thick red boundary around detected face
        cv2.polylines(frame, [best_contour], isClosed=True, color=(0, 0, 0), thickness=6)

        warped_face, M, dst_pts, ordered_pts = perspective_transform(frame, best_contour)
        warped_with_grid = draw_cubelet_grid(warped_face.copy())

        # Warp the grid image back to original perspective
        inv_M = cv2.getPerspectiveTransform(dst_pts, ordered_pts)
        overlay = cv2.warpPerspective(warped_with_grid, inv_M, (frame.shape[1], frame.shape[0]))

        # Create mask and apply only where grid exists
        mask = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
        frame[mask > 10] = overlay[mask > 10]

    cv2.imshow("Rubik's Cube Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[mjpeg @ 0x26eef880] overread 8


### Detecting the Colors in each cubelet

In [28]:
import cv2
import numpy as np

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 150)
    return edged

def find_largest_square_contour(edged):
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_approx = None
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > max_area:
                max_area = area
                best_approx = approx
    return best_approx

def draw_cubelets(frame, corners):
    if corners.shape != (4, 1, 2):
        return

    corners = corners.reshape((4, 2))
    top_left, top_right, bottom_right, bottom_left = corners

    def interp(p1, p2, t):
        return (1 - t) * np.array(p1) + t * np.array(p2)

    for i in range(1, 3):
        pt1 = tuple(interp(top_left, bottom_left, i / 3).astype(int))
        pt2 = tuple(interp(top_right, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

        pt1 = tuple(interp(top_left, top_right, i / 3).astype(int))
        pt2 = tuple(interp(bottom_left, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

def warp_perspective(frame, corners):
    corners = corners.reshape((4, 2))
    s = sum(corners)
    diff = np.diff(corners, axis=1)

    top_left = corners[np.argmin(s)]
    bottom_right = corners[np.argmax(s)]
    top_right = corners[np.argmin(diff)]
    bottom_left = corners[np.argmax(diff)]

    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    dst_pts = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype="float32")

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(frame, matrix, (300, 300))
    return warped

def get_cubelet_colors(warped):
    hsv = cv2.cvtColor(warped, cv2.COLOR_BGR2HSV)
    face_size = hsv.shape[0]
    cell_size = face_size // 3
    colors = []

    for row in range(3):
        for col in range(3):
            x = col * cell_size
            y = row * cell_size
            # Sample a small square from the center to avoid boundary interference
            center_offset = cell_size // 2
            sample_size = cell_size // 5  # small square
            x_center = x + center_offset
            y_center = y + center_offset

            cell = hsv[y_center - sample_size:y_center + sample_size,
                    x_center - sample_size:x_center + sample_size]

            avg_hsv = np.mean(cell.reshape(-1, 3), axis=0)

            hue = avg_hsv[0]
            sat = avg_hsv[1]
            val = avg_hsv[2]

            # Simple HSV-based color classification
            if val < 50:
                color = (0, 0, 0)         # black
            elif sat < 50:
                color = (200, 200, 200)   # white / gray
            elif hue < 10 or hue > 160:
                color = (0, 0, 255)       # red
            elif 10 < hue < 30:
                color = (0, 165, 255)     # orange
            elif 30 <= hue < 70:
                color = (0, 255, 0)       # green
            elif 70 <= hue < 100:
                color = (255, 255, 0)     # yellow
            elif 100 <= hue < 140:
                color = (255, 0, 0)       # blue
            else:
                color = (255, 255, 255)   # unknown - white

            colors.append(color)

    return colors


def draw_color_gui(frame, colors):
    gui_size = 150
    cell_size = gui_size // 3
    x_offset = frame.shape[1] - gui_size - 20
    y_offset = 20

    for i, color in enumerate(colors):
        row = i // 3
        col = i % 3
        top_left = (x_offset + col*cell_size, y_offset + row*cell_size)
        bottom_right = (top_left[0] + cell_size, top_left[1] + cell_size)

        cv2.rectangle(frame, top_left, bottom_right, color, -1)
        cv2.rectangle(frame, top_left, bottom_right, (0, 0, 0), 1)  # black border

    return frame

cap = cv2.VideoCapture(URL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, FRAME_SIZE)

    edged = preprocess_frame(frame)
    best_contour = find_largest_square_contour(edged)

    if best_contour is not None:
        cv2.polylines(frame, [best_contour], True, (0, 255, 0), 2)  # Red boundary for cube face
        draw_cubelets(frame, best_contour)
        warped_face = warp_perspective(frame, best_contour)

        colors = get_cubelet_colors(warped_face)
        frame = draw_color_gui(frame, colors)

    cv2.imshow("Rubik's Cube Detector", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np
import os

# Frame counter for saving
i = 1

# Reference RGB values for Rubik's cube colors
REFERENCE_COLORS = {
    'RED': (213, 48, 44),
    'BLUE': (10, 137, 192),
    'GREEN': (45, 190, 100),  # Adjusted GREEN to distinguish from BLUE
    'YELLOW': (220, 197, 83),
    'ORANGE': (209, 89, 37),
    'WHITE': (222, 218, 215)
}

# Create output directory if it doesn't exist
os.makedirs("captured", exist_ok=True)

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 150)
    return edged

def find_largest_square_contour(edged):
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_approx = None
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > max_area:
                max_area = area
                best_approx = approx
    return best_approx

def draw_cubelets(frame, corners):
    if corners.shape != (4, 1, 2):
        return

    corners = corners.reshape((4, 2))
    top_left, top_right, bottom_right, bottom_left = corners

    def interp(p1, p2, t):
        return (1 - t) * np.array(p1) + t * np.array(p2)

    for i in range(1, 3):
        pt1 = tuple(interp(top_left, bottom_left, i / 3).astype(int))
        pt2 = tuple(interp(top_right, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

        pt1 = tuple(interp(top_left, top_right, i / 3).astype(int))
        pt2 = tuple(interp(bottom_left, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

def warp_perspective(frame, corners):
    corners = corners.reshape((4, 2))
    s = sum(corners)
    diff = np.diff(corners, axis=1)

    top_left = corners[np.argmin(s)]
    bottom_right = corners[np.argmax(s)]
    top_right = corners[np.argmin(diff)]
    bottom_left = corners[np.argmax(diff)]

    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    dst_pts = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype="float32")

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(frame, matrix, (300, 300))
    return warped

def classify_color(rgb):
    min_dist = float('inf')
    closest_color = None
    for color_name, ref_rgb in REFERENCE_COLORS.items():
        dist = np.linalg.norm(np.array(rgb) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_color = color_name
    return closest_color

def get_cubelet_colors(warped_face):
    height, width, _ = warped_face.shape
    step_y, step_x = height // 3, width // 3
    labels = []

    for row in range(3):
        for col in range(3):
            y1 = row * step_y + step_y // 2 - 2
            y2 = row * step_y + step_y // 2 + 3
            x1 = col * step_x + step_x // 2 - 2
            x2 = col * step_x + step_x // 2 + 3

            roi = warped_face[y1:y2, x1:x2]
            avg_color = np.mean(roi, axis=(0, 1))  # BGR
            avg_color_rgb = avg_color[::-1]  # to RGB
            label = classify_color(avg_color_rgb)
            labels.append(label)

    return labels

def draw_color_gui(frame, color_labels):
    gui_size = 150
    cell_size = gui_size // 3
    x_offset = frame.shape[1] - gui_size - 20
    y_offset = 20

    for i, label in enumerate(color_labels):
        row = i // 3
        col = i % 3
        top_left = (x_offset + col * cell_size, y_offset + row * cell_size)
        bottom_right = (top_left[0] + cell_size, top_left[1] + cell_size)

        color_rgb = REFERENCE_COLORS.get(label, (255, 255, 255))
        color_bgr = tuple(int(c) for c in color_rgb[::-1])

        cv2.rectangle(frame, top_left, bottom_right, color_bgr, -1)
        cv2.rectangle(frame, top_left, bottom_right, (0, 0, 0), 1)

        # Draw text
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
        text_x = top_left[0] + (cell_size - text_size[0]) // 2
        text_y = top_left[1] + (cell_size + text_size[1]) // 2
        cv2.putText(frame, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    return frame

# Main capture loop
cap = cv2.VideoCapture(URL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, FRAME_SIZE)
    edged = preprocess_frame(frame)
    best_contour = find_largest_square_contour(edged)

    if best_contour is not None:
        warped_face = warp_perspective(frame, best_contour)
        cv2.polylines(frame, [best_contour], True, (0, 255, 0), 2)
        draw_cubelets(frame, best_contour)

        color_labels = get_cubelet_colors(warped_face)
        frame = draw_color_gui(frame, color_labels)

    cv2.imshow("Rubik's Cube Detector", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c') and best_contour is not None:
        cv2.imwrite(f"captured/frame_{i}.jpg", frame)
        cv2.imwrite(f"captured/warped_face_{i}.jpg", warped_face)
        print(f"[INFO] Saved frame_{i}.jpg and warped_face_{i}.jpg")
        i += 1

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[mjpeg @ 0x268d0f00] overread 8
[mjpeg @ 0x268d0f00] overread 8


In [10]:
import cv2
import numpy as np

REFERENCE_COLORS = {
    "WHITE": (255, 255, 255),
    "YELLOW": (255, 255, 0),
    "RED": (255, 0, 0),
    "GREEN": (0, 255, 0),
    "BLUE": (0, 0, 255),
    "ORANGE": (255, 165, 0)
}

RUBIKS_COLORS = {
    "WHITE": (255, 255, 255),
    "YELLOW": (255, 255, 0),
    "RED": (255, 0, 0),
    "GREEN": (0, 255, 0),
    "BLUE": (0, 0, 255),
    "ORANGE": (255, 165, 0)
}

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 150)
    return edged

def find_largest_square_contour(edged):
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_approx = None
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > max_area:
                max_area = area
                best_approx = approx
    return best_approx

def draw_cubelets(frame, corners):
    if corners.shape != (4, 1, 2):
        return

    corners = corners.reshape((4, 2))
    top_left, top_right, bottom_right, bottom_left = corners

    def interp(p1, p2, t):
        return (1 - t) * np.array(p1) + t * np.array(p2)

    for i in range(1, 3):
        pt1 = tuple(interp(top_left, bottom_left, i / 3).astype(int))
        pt2 = tuple(interp(top_right, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

        pt1 = tuple(interp(top_left, top_right, i / 3).astype(int))
        pt2 = tuple(interp(bottom_left, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

def warp_perspective(frame, corners):
    corners = corners.reshape((4, 2))
    s = corners.sum(axis=1)
    diff = np.diff(corners, axis=1)

    top_left = corners[np.argmin(s)]
    bottom_right = corners[np.argmax(s)]
    top_right = corners[np.argmin(diff)]
    bottom_left = corners[np.argmax(diff)]

    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    dst_pts = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype="float32")

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(frame, matrix, (300, 300))
    return warped

def classify_color(rgb):
    min_dist = float('inf')
    closest_color = None
    for color_name, ref_rgb in REFERENCE_COLORS.items():
        dist = np.linalg.norm(np.array(rgb) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_color = color_name
    return closest_color

def get_cubelet_colors(warped_face):
    height, width, _ = warped_face.shape
    step_y, step_x = height // 3, width // 3
    colors = []

    for row in range(3):
        row_colors = []
        for col in range(3):
            y1 = row * step_y + step_y // 2 - 2
            y2 = row * step_y + step_y // 2 + 3
            x1 = col * step_x + step_x // 2 - 2
            x2 = col * step_x + step_x // 2 + 3

            roi = warped_face[y1:y2, x1:x2]
            avg_color = np.mean(roi, axis=(0, 1))
            avg_color = avg_color[::-1]  # BGR to RGB
            label = classify_color(avg_color)
            row_colors.append(label)
        colors.append(row_colors)
    return colors

def closest_rubik_color(rgb):
    min_dist = float('inf')
    closest_color_name = 'UNKNOWN'
    for name, ref_rgb in REFERENCE_COLORS.items():
        dist = np.linalg.norm(np.array(rgb) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_color_name = name
    return closest_color_name

def draw_color_gui(frame, colors):
    gui_size = 150
    cell_size = gui_size // 3
    x_offset = frame.shape[1] - gui_size - 20
    y_offset = 20

    for row, row_colors in enumerate(colors):
        for col, color_name in enumerate(row_colors):
            top_left = (x_offset + col * cell_size, y_offset + row * cell_size)
            bottom_right = (top_left[0] + cell_size, top_left[1] + cell_size)

            # Draw filled rectangle with color
            color = RUBIKS_COLORS.get(color_name, (0, 0, 0))
            color = tuple(reversed(color))  # Convert RGB to BGR
            cv2.rectangle(frame, top_left, bottom_right, color, -1)
            cv2.rectangle(frame, top_left, bottom_right, (0, 0, 0), 1)  # black border

            # Draw text at center of each cell
            text_size = cv2.getTextSize(color_name, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
            text_x = top_left[0] + (cell_size - text_size[0]) // 2
            text_y = top_left[1] + (cell_size + text_size[1]) // 2
            cv2.putText(frame, color_name, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    return frame

cap = cv2.VideoCapture(URL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, FRAME_SIZE)

    edged = preprocess_frame(frame)
    best_contour = find_largest_square_contour(edged)

    if best_contour is not None:
        warped_face = warp_perspective(frame, best_contour)
        cv2.polylines(frame, [best_contour], True, (0, 255, 0), 2)  # Green boundary for cube face
        draw_cubelets(frame, best_contour)

        colors = get_cubelet_colors(warped_face)
        frame = draw_color_gui(frame, colors)

    cv2.imshow("Rubik's Cube Detector", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        cv2.imwrite(f"captured/frame_{i}.jpg", frame)
        cv2.imwrite(f"captured/warped_face_{i}.jpg", warped_face)
        print(f"[INFO] Saved frame_{i}.jpg and warped_face_{i}.jpg")
        i += 1

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[mjpeg @ 0x26c32880] overread 8
[mjpeg @ 0x26c32880] overread 8
[mjpeg @ 0x26c32880] overread 8
[mjpeg @ 0x26c32880] overread 8
[mjpeg @ 0x26c32880] overread 8
[mjpeg @ 0x26c32880] overread 7
[mjpeg @ 0x26c32880] overread 8


In [23]:
import cv2
import numpy as np

REFERENCE_COLORS = {
    "WHITE": (255, 255, 255),
    "YELLOW": (255, 255, 0),
    "RED": (255, 0, 0),
    "GREEN": (0, 255, 0),
    "BLUE": (0, 0, 255),
    "ORANGE": (255, 165, 0)
}

RUBIKS_COLORS = {
    "WHITE": (255, 255, 255),
    "YELLOW": (255, 255, 0),
    "RED": (255, 0, 0),
    "GREEN": (0, 255, 0),
    "BLUE": (0, 0, 255),
    "ORANGE": (255, 165, 0)
}

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 150)

    # Apply morphological closing to connect edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    edged = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

    return edged

def find_largest_square_contour(edged, min_area=10000):
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_approx = None
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > max_area and area >= min_area:
                max_area = area
                best_approx = approx
    return best_approx


def draw_cubelets(frame, corners):
    if corners.shape != (4, 1, 2):
        return

    corners = corners.reshape((4, 2))
    top_left, top_right, bottom_right, bottom_left = corners

    def interp(p1, p2, t):
        return (1 - t) * np.array(p1) + t * np.array(p2)

    for i in range(1, 3):
        pt1 = tuple(interp(top_left, bottom_left, i / 3).astype(int))
        pt2 = tuple(interp(top_right, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

        pt1 = tuple(interp(top_left, top_right, i / 3).astype(int))
        pt2 = tuple(interp(bottom_left, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

def warp_perspective(frame, corners):
    corners = corners.reshape((4, 2))
    s = corners.sum(axis=1)
    diff = np.diff(corners, axis=1)

    top_left = corners[np.argmin(s)]
    bottom_right = corners[np.argmax(s)]
    top_right = corners[np.argmin(diff)]
    bottom_left = corners[np.argmax(diff)]

    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    dst_pts = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype="float32")

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(frame, matrix, (300, 300))
    return warped

def classify_color(rgb):
    min_dist = float('inf')
    closest_color = None
    for color_name, ref_rgb in REFERENCE_COLORS.items():
        dist = np.linalg.norm(np.array(rgb) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_color = color_name
    return closest_color

def get_cubelet_colors(warped_face):
    height, width, _ = warped_face.shape
    step_y, step_x = height // 3, width // 3
    colors = []

    for row in range(3):
        row_colors = []
        for col in range(3):
            y1 = row * step_y + step_y // 2 - 2
            y2 = row * step_y + step_y // 2 + 3
            x1 = col * step_x + step_x // 2 - 2
            x2 = col * step_x + step_x // 2 + 3

            roi = warped_face[y1:y2, x1:x2]
            avg_color = np.mean(roi, axis=(0, 1))
            avg_color = avg_color[::-1]  # BGR to RGB
            label = classify_color(avg_color)
            row_colors.append(label)
        colors.append(row_colors)
    return colors

def closest_rubik_color(rgb):
    min_dist = float('inf')
    closest_color_name = 'UNKNOWN'
    for name, ref_rgb in REFERENCE_COLORS.items():
        dist = np.linalg.norm(np.array(rgb) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_color_name = name
    return closest_color_name

def draw_color_gui(frame, colors):
    gui_size = 150
    cell_size = gui_size // 3
    x_offset = frame.shape[1] - gui_size - 20
    y_offset = 20

    for row, row_colors in enumerate(colors):
        for col, color_name in enumerate(row_colors):
            top_left = (x_offset + col * cell_size, y_offset + row * cell_size)
            bottom_right = (top_left[0] + cell_size, top_left[1] + cell_size)

            # Draw filled rectangle with color
            color = RUBIKS_COLORS.get(color_name, (0, 0, 0))
            color = tuple(reversed(color))  # Convert RGB to BGR
            cv2.rectangle(frame, top_left, bottom_right, color, -1)
            cv2.rectangle(frame, top_left, bottom_right, (0, 0, 0), 1)  # black border

            # Draw text at center of each cell
            text_size = cv2.getTextSize(color_name, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
            text_x = top_left[0] + (cell_size - text_size[0]) // 2
            text_y = top_left[1] + (cell_size + text_size[1]) // 2
            cv2.putText(frame, color_name, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    return frame

cap = cv2.VideoCapture(URL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, FRAME_SIZE)

    edged = preprocess_frame(frame)
    best_contour = find_largest_square_contour(edged)

    if best_contour is not None:
        warped_face = warp_perspective(frame, best_contour)
        cv2.polylines(frame, [best_contour], True, (0, 255, 0), 2)  # Green boundary for cube face
        draw_cubelets(frame, best_contour)

        colors = get_cubelet_colors(warped_face)
        frame = draw_color_gui(frame, colors)

    cv2.imshow("Rubik's Cube Detector", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        cv2.imwrite(f"captured/frame_{i}.jpg", frame)
        cv2.imwrite(f"captured/warped_face_{i}.jpg", warped_face)
        print(f"[INFO] Saved frame_{i}.jpg and warped_face_{i}.jpg")
        i += 1

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Calibration

In [35]:
import cv2
import numpy as np
import json

# Approximate BGR reference values of Rubik’s Cube center colors (used for UI)
REFERENCE_COLORS = {
    "WHITE": (222, 218, 215),
    "YELLOW": (220, 197, 83),
    "RED": (213, 48, 44),
    "GREEN": (75, 189, 75),
    "BLUE": (10, 137, 192),
    "ORANGE": (209, 89, 37)
}

# Use color names instead of face positions
COLOR_ORDER = ["WHITE", "RED", "BLUE", "YELLOW", "ORANGE", "GREEN"]
SAVE_PATH = "cube_calibration.json"

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    return cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

def find_largest_square_contour(edged, min_area=10000):
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area, best_approx = 0, None
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > max_area and area >= min_area:
                max_area = area
                best_approx = approx
    return best_approx

def warp_perspective(frame, corners):
    corners = corners.reshape((4, 2))
    s = corners.sum(axis=1)
    diff = np.diff(corners, axis=1)
    top_left = corners[np.argmin(s)]
    bottom_right = corners[np.argmax(s)]
    top_right = corners[np.argmin(diff)]
    bottom_left = corners[np.argmax(diff)]

    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    dst_pts = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype="float32")

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    return cv2.warpPerspective(frame, matrix, (300, 300))

def get_center_hsv(warped_face):
    hsv_face = cv2.cvtColor(warped_face, cv2.COLOR_BGR2HSV)
    h, w = hsv_face.shape[:2]
    cell_size = h // 3
    hsv_values = []
    for row in range(3):
        for col in range(3):
            cx = col * cell_size + cell_size // 2
            cy = row * cell_size + cell_size // 2
            roi = hsv_face[cy-2:cy+3, cx-2:cx+3]
            avg_hsv = np.mean(roi.reshape(-1, 3), axis=0)
            hsv_values.append(avg_hsv.tolist())
    return hsv_values

def calibrate_cube():
    cap = cv2.VideoCapture(URL)
    calibration = {}

    for color_name in COLOR_ORDER:
        print(f"\n➡️  Show the '{color_name}' face (center should be {color_name}). Press SPACE to capture.")

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, FRAME_SIZE)
            edged = preprocess_frame(frame)
            contour = find_largest_square_contour(edged)

            if contour is not None:
                cv2.polylines(frame, [contour], True, (0, 255, 0), 2)

            cv2.putText(frame, f"Show {color_name} face, press SPACE to capture", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow("Calibration", frame)

            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC to exit
                cap.release()
                cv2.destroyAllWindows()
                return
            if key == 32 and contour is not None:  # SPACE to capture
                warped = warp_perspective(frame, contour)
                calibration[color_name] = get_center_hsv(warped)
                print(f"✅ Captured {color_name} face.")
                break

    cap.release()
    cv2.destroyAllWindows()

    with open(SAVE_PATH, "w") as f:
        json.dump(calibration, f, indent=2)
    print(f"\n✅ Calibration complete. Saved to {SAVE_PATH}")

calibrate_cube()


➡️  Show the 'WHITE' face (center should be WHITE). Press SPACE to capture.
✅ Captured WHITE face.

➡️  Show the 'RED' face (center should be RED). Press SPACE to capture.
✅ Captured RED face.

➡️  Show the 'BLUE' face (center should be BLUE). Press SPACE to capture.
✅ Captured BLUE face.

➡️  Show the 'YELLOW' face (center should be YELLOW). Press SPACE to capture.
✅ Captured YELLOW face.

➡️  Show the 'ORANGE' face (center should be ORANGE). Press SPACE to capture.
✅ Captured ORANGE face.

➡️  Show the 'GREEN' face (center should be GREEN). Press SPACE to capture.
✅ Captured GREEN face.

✅ Calibration complete. Saved to cube_calibration.json


In [36]:
import cv2
import numpy as np
import json

# Load calibration HSV data from file
with open("cube_calibration.json", "r") as f:
    calibration_data = json.load(f)

# Compute average HSV for each face
AVERAGE_HSV = {}
for face, values in calibration_data.items():
    hsv_array = np.array(values, dtype=np.float32)
    mean_hsv = np.mean(hsv_array, axis=0)
    AVERAGE_HSV[face] = mean_hsv

RUBIKS_COLORS = {
    "WHITE": (255, 255, 255),
    "YELLOW": (255, 255, 0),
    "RED": (255, 0, 0),
    "GREEN": (0, 255, 0),
    "BLUE": (0, 0, 255),
    "ORANGE": (255, 165, 0)
}

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blur, 50, 150)

    # Apply morphological closing to connect edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    edged = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

    return edged

def find_largest_square_contour(edged, min_area=10000):
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_approx = None
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            area = cv2.contourArea(approx)
            if area > max_area and area >= min_area:
                max_area = area
                best_approx = approx
    return best_approx


def draw_cubelets(frame, corners):
    if corners.shape != (4, 1, 2):
        return

    corners = corners.reshape((4, 2))
    top_left, top_right, bottom_right, bottom_left = corners

    def interp(p1, p2, t):
        return (1 - t) * np.array(p1) + t * np.array(p2)

    for i in range(1, 3):
        pt1 = tuple(interp(top_left, bottom_left, i / 3).astype(int))
        pt2 = tuple(interp(top_right, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

        pt1 = tuple(interp(top_left, top_right, i / 3).astype(int))
        pt2 = tuple(interp(bottom_left, bottom_right, i / 3).astype(int))
        cv2.line(frame, pt1, pt2, (0, 0, 0), 1)

def warp_perspective(frame, corners):
    corners = corners.reshape((4, 2))
    s = corners.sum(axis=1)
    diff = np.diff(corners, axis=1)

    top_left = corners[np.argmin(s)]
    bottom_right = corners[np.argmax(s)]
    top_right = corners[np.argmin(diff)]
    bottom_left = corners[np.argmax(diff)]

    src_pts = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    dst_pts = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype="float32")

    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(frame, matrix, (300, 300))
    return warped

def classify_color(hsv):
    min_dist = float('inf')
    closest_face = None
    for face, ref_hsv in AVERAGE_HSV.items():
        dist = np.linalg.norm(np.array(hsv) - np.array(ref_hsv))
        if dist < min_dist:
            min_dist = dist
            closest_face = face
    return closest_face


def get_cubelet_colors(warped_face):
    height, width, _ = warped_face.shape
    step_y, step_x = height // 3, width // 3
    colors = []

    for row in range(3):
        row_colors = []
        for col in range(3):
            y1 = row * step_y + step_y // 2 - 2
            y2 = row * step_y + step_y // 2 + 3
            x1 = col * step_x + step_x // 2 - 2
            x2 = col * step_x + step_x // 2 + 3

            roi = warped_face[y1:y2, x1:x2]
            roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            avg_hsv = np.mean(roi_hsv, axis=(0, 1))
            label = classify_color(avg_hsv)
            row_colors.append(label)
        colors.append(row_colors)
    return colors


def draw_color_gui(frame, colors):
    gui_size = 150
    cell_size = gui_size // 3
    x_offset = frame.shape[1] - gui_size - 20
    y_offset = 20

    for row, row_colors in enumerate(colors):
        for col, color_name in enumerate(row_colors):
            top_left = (x_offset + col * cell_size, y_offset + row * cell_size)
            bottom_right = (top_left[0] + cell_size, top_left[1] + cell_size)

            # Draw filled rectangle with color
            color = RUBIKS_COLORS.get(color_name, (0, 0, 0))
            color = tuple(reversed(color))  # Convert RGB to BGR
            cv2.rectangle(frame, top_left, bottom_right, color, -1)
            cv2.rectangle(frame, top_left, bottom_right, (0, 0, 0), 1)  # black border

            # Draw text at center of each cell
            text_size = cv2.getTextSize(color_name, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
            text_x = top_left[0] + (cell_size - text_size[0]) // 2
            text_y = top_left[1] + (cell_size + text_size[1]) // 2
            cv2.putText(frame, color_name, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    return frame

cap = cv2.VideoCapture(URL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, FRAME_SIZE)

    edged = preprocess_frame(frame)
    best_contour = find_largest_square_contour(edged)

    if best_contour is not None:
        warped_face = warp_perspective(frame, best_contour)
        cv2.polylines(frame, [best_contour], True, (0, 255, 0), 2)  # Green boundary for cube face
        draw_cubelets(frame, best_contour)

        colors = get_cubelet_colors(warped_face)
        frame = draw_color_gui(frame, colors)

    cv2.imshow("Rubik's Cube Detector", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        cv2.imwrite(f"captured/frame_{i}.jpg", frame)
        cv2.imwrite(f"captured/warped_face_{i}.jpg", warped_face)
        print(f"[INFO] Saved frame_{i}.jpg and warped_face_{i}.jpg")
        i += 1

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
[mjpeg @ 0x26c8f780] overread 8
